### List datasets that will be used for training with scBasset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scbasset

2024-05-12 17:50:58.800205: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-12 17:50:59.089640: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-12 17:50:59.134695: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-12 17:50:59.134720: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [3]:
cd ~/workspace/theislab/mubind-pipeline/notebooks/pipeline

/mnt/c/Users/IgnacioIbarra/Dropbox/workspace/theislab/mubind-pipeline/notebooks/pipeline


In [4]:
# read scbasset
import tensorflow as tf
import scanpy as sc
from scbasset.utils import *
from scbasset.basenji_utils import *
from scbasset.utils import *
import pandas as pd
import glob
import os
import anndata


### Load scBasset models and score with them

In [57]:
rootdir = '/mnt/f/workspace/theislab/mubind/data/*/*/scbasset_output/obs*/*/*'

df = []

overwrite = True
for p in glob.glob(rootdir):

    # print(p)
    res = []
    # if 'organoids' in p:
    #     continue
    if not '/obs500_e10/' in p:
        continue

    if not os.path.exists(p):
        continue
    if not 'best_model.h5' in p:
        continue


    scores_path = p.replace('.h5', '_scores.csv')
    print(os.path.exists(scores_path), scores_path)
    if os.path.exists(scores_path) and not overwrite:
        df2 = pd.read_csv(scores_path, index_col=0)
        df2['n_obs'] = int(scores_path.split('/')[-3].replace('obs', '').split('_')[0])
        df.append(df2)
        continue

    print(p)

    os.makedirs("results", exist_ok=True)

    n_obs = p.split('/')[-3].split('_')[0]
    dataset_name = p.split('/')[-6]
    feature_selection_rule = p.split('/')[-5]
    loss_key = p.split('/')[-2]

    ad_path = os.path.join(os.path.dirname(p),
                            '../../../scbasset_input/%s/ad.h5ad' % n_obs)
    seqs_path = os.path.join(os.path.dirname(p),
                                '../../../scbasset_input/%s/all_seqs.h5' % n_obs)
    train_seqs = os.path.join(os.path.dirname(p),
                                '../../../scbasset_input/%s/train_seqs.h5' % n_obs)
    val_seqs = os.path.join(os.path.dirname(p),
                                '../../../scbasset_input/%s/val_seqs.h5' % n_obs)
    test_seqs = os.path.join(os.path.dirname(p),
                                '../../../scbasset_input/%s/test_seqs.h5' % n_obs)
    splits_path = os.path.join(os.path.dirname(p),
                                '../../../scbasset_input/%s/splits.h5' % n_obs)
    print(os.path.exists(ad_path), ad_path)

    # read h5ad file
    ad = anndata.read_h5ad(ad_path)

    # load model
    model = make_model(32, ad.shape[0], show_summary=False)
    model.load_weights(p)


    # since imputation generates a dense cell by peak matrix as output.
    # it won't scale to very large dataset due to memory issue.
    # predict scores

    print('get sequences scores')
    all_seqs = seqs_path # from the preprocess step
    # make tesorflow dataset to feed into model
    m = ad.X.tocoo().transpose().tocsr()
    n_cells = ad.shape[0]
    all_ds = tf.data.Dataset.from_generator(
        generator(all_seqs, m), 
        output_signature=(
                tf.TensorSpec(shape=(1344,4), dtype=tf.int8),
                tf.TensorSpec(shape=(n_cells), dtype=tf.int8),
        )
    ).batch(128).prefetch(tf.data.AUTOTUNE)
    
    # Y_impute = imputation_Y_normalize(all_ds, model)
    Y_pred = model.predict(all_ds)
    Y_pred = Y_pred.T

    Y_true = ad.X.A

    # convert to csr matrix
    import h5py
    with h5py.File(splits_path, 'r') as hf:
        train_ids = hf['train_ids'][:]
        val_ids = hf['val_ids'][:]
        test_ids = hf['test_ids'][:]

    import matplotlib.pyplot as plt
    from sklearn.metrics import r2_score, roc_auc_score
    from sklearn.metrics import average_precision_score
    # plt.scatter(Y_true.A.flatten(), Y_pred.flatten())


    for next_ids, label in zip([train_ids, val_ids, test_ids], ['train', 'val', 'test']):

        # roc auc

        Y_true_roc = np.where(Y_true[:,next_ids] > 0, 1, Y_true[:,next_ids])
        
        roc_auc, pr_auc = np.nan, np.nan
        try:
            roc_auc = roc_auc_score(Y_true_roc.flatten(), Y_pred[:,next_ids].flatten())
            pr_auc = average_precision_score(Y_true_roc.flatten(), Y_pred[:,next_ids].flatten())
        except:
            print('problem while calculating PR/AUC (maybe full zero mats)')

        res.append([dataset_name, feature_selection_rule, loss_key, label, 'roc_auc', roc_auc, p])
        res.append([dataset_name, feature_selection_rule, loss_key, label, 'pr_auc', pr_auc, p])
        
        r2 = r2_score(Y_true[:,next_ids].flatten(), Y_pred[:,next_ids].flatten())
        res.append([dataset_name, feature_selection_rule, loss_key, label, 'r2', r2, p])        
        
    
    res = pd.DataFrame(res, columns=['dataset', 'feat_selection', 'loss_key', 'group', 'metric', 'value', p])
    res.to_csv(scores_path)

    df.append(res)

True /mnt/f/workspace/theislab/mubind/data/noack_2022/random/scbasset_output/obs500_e10/poisson/best_model_scores.csv
/mnt/f/workspace/theislab/mubind/data/noack_2022/random/scbasset_output/obs500_e10/poisson/best_model.h5
True /mnt/f/workspace/theislab/mubind/data/noack_2022/random/scbasset_output/obs500_e10/poisson/../../../scbasset_input/obs500/ad.h5ad
get sequences scores
12/12 [==============================] - 6s 484ms/step
True /mnt/f/workspace/theislab/mubind/data/noack_2022/random/scbasset_output/obs500_e10/bce/best_model_scores.csv
/mnt/f/workspace/theislab/mubind/data/noack_2022/random/scbasset_output/obs500_e10/bce/best_model.h5
True /mnt/f/workspace/theislab/mubind/data/noack_2022/random/scbasset_output/obs500_e10/bce/../../../scbasset_input/obs500/ad.h5ad
get sequences scores
12/12 [==============================] - 6s 485ms/step
True /mnt/f/workspace/theislab/mubind/data/noack_2022/episcanpy/scbasset_output/obs500_e10/poisson/best_model_scores.csv
/mnt/f/workspace/theisl

In [51]:
res = pd.concat(df)
res['model'] = 'scBasset'
print('')

In [52]:
res[res['metric'] == 'roc_auc']

,dataset,feat_selection,loss_key,group,metric,value,model
0,noack_2022,random,poisson,train,roc_auc,0.725056,scBasset
3,noack_2022,random,poisson,val,roc_auc,0.674981,scBasset
6,noack_2022,random,poisson,test,roc_auc,0.635594,scBasset
0,noack_2022,random,bce,train,roc_auc,0.665473,scBasset
3,noack_2022,random,bce,val,roc_auc,0.638341,scBasset
6,noack_2022,random,bce,test,roc_auc,0.622321,scBasset
0,noack_2022,episcanpy,poisson,train,roc_auc,NaN,scBasset
3,noack_2022,episcanpy,poisson,val,roc_auc,NaN,scBasset
6,noack_2022,episcanpy,poisson,test,roc_auc,NaN,scBasset
0,noack_2022,episcanpy,bce,train,roc_auc,NaN,scBasset


In [54]:
res.to_csv('results_scbasset.csv')

In [55]:
print('done...')

done...
